# Deckbuild Inference

Build optimal decks from card pools using a trained model.

The algorithm has two phases:
1. **Mean-field**: Iteratively refine fractional card counts until convergence
2. **Card-by-card**: Greedily swap weak deck cards for strong sideboard cards

In [ ]:
import sys
sys.path.insert(0, '..')

import random
import torch
import statisticaldeckbuild as sdb

In [ ]:
# Configuration
SET_ABBREVIATION = "FDN"
DRAFT_MODE = "Premier"

In [ ]:
# Initialize the deck builder
builder = sdb.IterativeDeckBuilder(
    set_abbreviation=SET_ABBREVIATION,
    draft_mode=DRAFT_MODE,
    model_folder="../data/models/",
    cards_folder="../data/cards/",
)
print(f"Loaded model for {SET_ABBREVIATION} {DRAFT_MODE}")
print(f"Card pool: {builder.num_cards} cards")

In [ ]:
# Load validation dataset to get real draft pools
val_path = f"../data/training_sets/{SET_ABBREVIATION}_{DRAFT_MODE}_deckbuild_val.pth"
val_dataset = torch.load(val_path, weights_only=False)
print(f"Loaded {len(val_dataset)} validation examples")

In [ ]:
def get_pool_from_dataset(dataset, index):
    """Extract the full card pool (deck + sideboard) from a dataset example."""
    deck = dataset.decks[index]
    sideboard = dataset.sideboards[index]
    pool = []
    for i, (deck_count, sb_count) in enumerate(zip(deck, sideboard)):
        card_name = dataset.cardnames[i]
        pool.extend([card_name] * int(deck_count + sb_count))
    return pool

## Build a Deck with Verbose Output

Watch both phases of the algorithm in action.

In [ ]:
# Pick a random example
random_idx = random.randint(0, len(val_dataset) - 1)
pool = get_pool_from_dataset(val_dataset, random_idx)
print(f"Example #{random_idx}: Pool has {len(pool)} cards\n")

# Build deck with verbose output
result = builder.build_deck(pool, target_deck_size=23, verbose=True)

## Phase 1: Mean-Field Result

Shows the fractional deck after mean-field convergence. Cards near 100% are strong includes, cards near 0% are sideboard material.

In [ ]:
# Show mean-field fractional deck
builder.print_mean_field_deck(result)

## Phase 2: Card-by-Card Swaps

After rounding the mean-field result, we greedily swap the weakest deck card with the strongest sideboard card until no beneficial swaps remain.

In [ ]:
# Show swap history
if result['swap_history']:
    print("Card-by-card swap history:")
    print(f"{'#':>3}  {'Card Out':<30} {'Score':>6}  ->  {'Card In':<30} {'Score':>6}")
    print("-" * 90)
    for i, (card_out, card_in, score_out, score_in) in enumerate(result['swap_history'], 1):
        print(f"{i:>3}  {card_out:<30} {score_out:>6.2f}  ->  {card_in:<30} {score_in:>6.2f}")
else:
    print("No swaps needed - mean-field rounding produced optimal deck!")

## Final Deck and Sideboard

In [ ]:
# Show final deck and sideboard
builder.print_deck_and_sideboard(result, pool)

## Quick Build (No Verbose)

For production use, just call `build_deck` without verbose.

In [ ]:
# Quick build on a few random pools
for _ in range(3):
    idx = random.randint(0, len(val_dataset) - 1)
    pool = get_pool_from_dataset(val_dataset, idx)
    result = builder.build_deck(pool, target_deck_size=23, verbose=False)
    
    print(f"\nExample #{idx}: {len(pool)} cards in pool")
    print(f"  Mean-field: {result['mean_field_iterations']} iterations")
    print(f"  Card-by-card: {result['card_by_card_swaps']} swaps")
    print(f"  Deck: {len(result['deck'])} cards, Sideboard: {len(result['sideboard'])} cards")